### Generate Inpainted Dataset

Code that inpaints a given dataset, before running have StableDiffusion runnning its API version on http://127.0.0.1:7861

In [1]:
import base64
import requests
import os
import copy
import cv2
import json
import numpy as np

def imgJson(src):
    with open(src, "rb") as f:
        img = f.read()
        
    return base64.b64encode(img).decode("utf8")

def jsonImg(img):
    jpg_original = base64.b64decode(img)
    jpg_as_np = np.frombuffer(jpg_original, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

def inpaint(img_path, mask_path, save_path, url = "http://127.0.0.1:7861"):

    img = imgJson(img_path)
    mask = imgJson(mask_path)

    # prompt = "photo of one real (chair), photorealistic, hyper-realistic, hyper-detailed"
    prompt = "rendering of (one) 3D (chair), render, hyper realistic"
    negative_prompt = "text, logo"

    payload = {
        "init_images": [
        img
        ],
        "resize_mode": 0, # Just resize:0, crop_and_resize:1, resize_and_fill:2
        "denoising_strength": 0.9,
        "mask": mask,
        "mask_blur": 0,
        "inpainting_fill": 2, # fill:0, original:1, noise:2, latent nothing:3
        "inpaint_full_res": True,
        "inpaint_full_res_padding": 64,
        "inpainting_mask_invert": 1,
        "prompt": prompt,
        "seed": -1,
        "sampler_name": "Euler a",
        "sampler_index": 0, # Euler a
        "batch_size": 1,
        "n_iter": 1,
        "steps": 10,
        "cfg_scale": 7,
        "width": 512,
        "height": 512,
        "restore_faces": False,
        "tiling": False,
        "negative_prompt": negative_prompt,
        "include_init_images": False,
    }

    file_settings = save_path+'/settings.txt'
    if not os.path.exists(file_settings):
        temp = copy.deepcopy(payload)
        del temp["init_images"]
        del temp["mask"]

        with open(file_settings, 'w') as file:
            file.write(json.dumps(temp)) 

    response = requests.post(url=f'{url}/sdapi/v1/img2img', json=payload)

    r = response.json()
    # print(r)
    resized = cv2.resize(jsonImg(r["images"][0]), (137,137), interpolation = cv2.INTER_AREA)
    
    return resized

def inpaintDataset(renders_path, output_path, taxonomy_file, url = "http://127.0.0.1:7861"):
    
    class_id = "03001627"

    taxonomies = []
    with open(taxonomy_file, encoding='utf-8') as file:
        taxonomies = json.loads(file.read())

    taxonomies = {t['taxonomy_id']: t for t in taxonomies}

    model_ids = taxonomies[class_id]["test"]

    for sample_idx, sample_name in enumerate(model_ids):
        
        if(sample_idx%100==0):
            print("=== %04d/%04d === %s" %(sample_idx,len(model_ids),sample_name))

        output_sample_path = os.path.join(output_path, class_id, sample_name, "rendering")

        if not os.path.exists(output_sample_path):
            os.makedirs(output_sample_path)
            
        # Get file list of rendering images
        image_idx = 0
        
        file_path = renders_path % (class_id, sample_name)
        img_file_path = os.path.join(file_path, "%02d.png"%image_idx)

        # Inpaint
        mask_path = os.path.join(file_path, '%02d_mask.png'%image_idx)
        save_img = os.path.join(output_sample_path, '%02d.png'%image_idx)
        inpainted = inpaint(img_file_path,mask_path,save_img,output_sample_path)
        cv2.imwrite(save_img, inpainted)   

In [ ]:
erasing_percentages = [15,25,50]
url = "http://127.0.0.1:7861"

for erasing in erasing_percentages:

    print(f"=== ORE_Center_{erasing}")

    taxonomy_file = "./ShapeNet_Chair.json"
    occluded_path = f'./ORE_Center_{erasing}'+'/%s/%s/rendering/'
    output_path = f'./Inpainting/ORE_Center_Inpainting_{erasing}'

    inpaintDataset(occluded_path, output_path, taxonomy_file, url)

### Generate Combined Version

In [ ]:
import os
import cv2
import json
import numpy as np
from PIL import Image
import shutil

for m in [15,25,50]:

    taxonomy_file = "/home/davide/Desktop/Dissertation/Models/Pix2Vox/datasets/ShapeNet_Chair.json"
    inpainting_path = f'/home/davide/Desktop/Dissertation/Data/OccludedShapeNet/Inpainted/ORE_Random_Inpainting_{m}'+'/%s/%s/rendering/%02d.png'
    occluded_path = f'/home/davide/Desktop/Dissertation/Data/OccludedShapeNet/ORE_Random_{m}'+'/%s/%s/rendering/%02d.png'
    output_path= f'/home/davide/Desktop/Dissertation/Data/OccludedShapeNet/Combined/ORE_Random_Combined_{m}/'

    class_id = "03001627"

    taxonomies = []
    with open(taxonomy_file, encoding='utf-8') as file:
        taxonomies = json.loads(file.read())

    taxonomies = {t['taxonomy_id']: t for t in taxonomies}
    model_ids = taxonomies[class_id]["test"]

    for sample_idx, sample_name in enumerate(model_ids):

        print("=== %04d/%04d === %s" %(sample_idx,len(model_ids),sample_name))

        output_sample_path = os.path.join(output_path, class_id, sample_name, "rendering")

        if not os.path.exists(output_sample_path):
            os.makedirs(output_sample_path)
            
        # inpainting
        img_inpainting_path = inpainting_path % (class_id, sample_name, 0)
        # occluded
        img_occluded_path = occluded_path % (class_id, sample_name, 0)

        if os.path.exists(img_inpainting_path) and os.path.exists(img_occluded_path):
            dst = os.path.join(output_sample_path, "00.png")
            # shutil.copy(img_inpainting_path, dst)

            img = cv2.imread(img_inpainting_path, cv2.IMREAD_UNCHANGED)
            b_channel, g_channel, r_channel = cv2.split(img)
            alpha_channel = np.ones(b_channel.shape, dtype=b_channel.dtype) * 255 #creating a dummy alpha channel image.
            img_BGRA = cv2.merge((b_channel, g_channel, r_channel, alpha_channel))

            cv2.imwrite(dst,img_BGRA)

            dst = os.path.join(output_sample_path, "01.png")
            shutil.copy(img_occluded_path, dst)